uncomment if executing in google colab

In [1]:
# ! pip uninstall pyro-ppl
# ! pip install pyro-ppl==1.5.1
# ! unzip fonts.zip 
# ! unzip claptchagen.zip
# ! unzip csis
# ! cp csis.py /usr/local/lib/python3.7/dist-packages/pyro/infer

In [2]:
import random
import string
import os
import torch.nn as nn
import torch
import torch.nn.functional as F
import pyro
import numpy as np
import pyro.optim as optim
import pyro.distributions as dist
import pyro.infer
import pyro.optim

from torch.autograd import Variable
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO
from PIL import Image
from claptchagen.claptcha import Claptcha
from torch.distributions import constraints
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
print(pyro.__version__)

captcha_folder = 'generated_captchas'
captchaHeight = 32
captchaWidth = 100
captchaMarginX = 4
captchaMarginY = 4
batch_size = 10

#char_dict = string.ascii_lowercase
char_dict = string.digits
#char_dict = "abc"
USE_CUDA = True
MAX_N = 5 # maximum number of letters in a captcha 
MIN_N = 3 # minimum number of letters in a captcha
MIN_NOISE = 0.01 # minimum noise
MAX_NOISE = 0.99 # maximum noise
smoke_test = False
num_steps = 200 if not smoke_test else 10
TrainingSample = 10000 if not smoke_test else 100 # number of captchas generated for training

def randomString():
    """
    return a string with <num_char> random letters
    """
    k = random.randint(MIN_N, MAX_N) # sample number of characters
    
    rndLetters = (random.choice(char_dict) for _ in range(k))
    
    pad_spaces = MAX_N - k # pad the string so the captcha is close to center
    space = " " * (pad_spaces // 2)
    
    return space + "".join(rndLetters) + space

def ramdomNoise():
    """
    return a float between MIN_NOISE, MAX_NOISE
    """
    return random.uniform(MIN_NOISE, MAX_NOISE)

def generate_random_captcha(n, save=False):
    """
    generate n random captchas,
    return a list of texts on the captchas
    """
    # Initialize Claptcha object with random text, FreeMono as font, of size
    # 100x30px, using bicubic resampling filter and adding a bit of white noise
    c = Claptcha(randomString, "fonts/FreeSans.ttf", (captchaWidth, captchaHeight), (captchaMarginX, captchaMarginY),
             resample=Image.BILINEAR, noise=0)
    captcha_generated = [ [] for i in range(MAX_N)]
    for i in range(n):
        c.noise = ramdomNoise()
        if save:
            text, _ = c.write(os.path.join(captcha_folder, 'captcha{}.png'.format(i)))
            os.rename(os.path.join(captcha_folder, 'captcha{}.png'.format(i)),os.path.join(captcha_folder, '{}.png'.format(text + "_" + str(i))))
        text, image = c.image
        text = text.strip()
        image = np.array(image)[:, :, 0] # the generator is gray scale, only keep one channel is enough
        captcha_generated[len(text) - 1].append((text, image, c.noise))
    return captcha_generated
    
captcha_generated = generate_random_captcha(TrainingSample, save=False)
for lst in captcha_generated:
    print("number of samples in group", len(lst))
    # print some sample captcha information generated
    for i, t in enumerate(lst):
        print("text", t[0], "captcha shape", t[1].shape, "noise", t[2])
        if i >= 10:
            break

1.5.1
number of samples in group 0
number of samples in group 0
number of samples in group 3365
text 242 captcha shape (32, 100) noise 0.03777901895821142
text 729 captcha shape (32, 100) noise 0.05425382710690266
text 671 captcha shape (32, 100) noise 0.2302128383592112
text 094 captcha shape (32, 100) noise 0.8648025370445467
text 812 captcha shape (32, 100) noise 0.3536950630496512
text 269 captcha shape (32, 100) noise 0.7249930206252385
text 469 captcha shape (32, 100) noise 0.7983828489554744
text 995 captcha shape (32, 100) noise 0.3666206254469832
text 760 captcha shape (32, 100) noise 0.033784325056583304
text 021 captcha shape (32, 100) noise 0.32419312558370855
text 668 captcha shape (32, 100) noise 0.3836148905552011
number of samples in group 3331
text 5093 captcha shape (32, 100) noise 0.5089604957145272
text 3833 captcha shape (32, 100) noise 0.5362363504654207
text 8036 captcha shape (32, 100) noise 0.9331880606851088
text 2275 captcha shape (32, 100) noise 0.4799116745

In [3]:
def render_image(chars, fonts="fonts/FreeSans.ttf", size=(captchaWidth, captchaHeight), 
                 margin=(captchaMarginX, captchaMarginY), resample=Image.BILINEAR, noise=0.3, use_cuda=False):
    #noise = noise.data.item()
    #print(chars, noise)
    pad_spaces = MAX_N - len(chars)
    space = " " * (pad_spaces // 2)
    chars = space + chars + space
    render = Claptcha(chars, fonts, size, margin, resample=resample, noise=noise)

    
    _ , rendered_image = render.image
    rendered_image = np.array(rendered_image)[:,:,0] # the generator is gray scale, only keep one channel is enough
    rendered_image = np.subtract(np.divide(rendered_image, 255), 0.5)
    rendered_image = torch.from_numpy(rendered_image)
    if use_cuda:
        rendered_image = rendered_image.cuda()
    return rendered_image

In [4]:
class CaptchaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, raw_captchas, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.raw_captchas = raw_captchas
        self.transform = transform

    def __len__(self):
        return len(self.raw_captchas)

    def __getitem__(self, idx):
        label = self.raw_captchas[idx][0]
        image = self.raw_captchas[idx][1]
        noise = self.raw_captchas[idx][2]
        
        image = np.subtract(np.divide(image, 255), 0.5)
        image = torch.from_numpy(image).float()

        if self.transform:
            image = self.transform(image)

        return label, image, noise

In [5]:
def make_loarders(BATCH_SIZE, raw_samples):
    dataloaders = [] # dataloaders for different num of char
    for lst in raw_samples:
        if lst:
            ds = CaptchaDataset(lst)
            dataloader = DataLoader(ds, batch_size=BATCH_SIZE,
                                    shuffle=True, num_workers=0, drop_last=True)
            dataloaders.append(dataloader)
    return dataloaders

def make_batches(dataloaders):
    all_batches = []
    for dl in dataloaders:
        for i_batch, sample in enumerate(dl):
            all_batches.append(sample)
    random.shuffle(all_batches)
    random.shuffle(all_batches)
    return all_batches

TrainLoaders = make_loarders(BATCH_SIZE=batch_size, raw_samples=captcha_generated)

In [6]:
class NoiseNet(nn.Module):

    def __init__(self, img_size, out_size = 1):
        """
        Network for learning noise in a captcha
        """
        super(NoiseNet, self).__init__()
        
        self.img_size = img_size
        self.fc0 = nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1])
        self.fc1 = nn.Linear(img_size[0] * img_size[1], 1024)

        self.fc21 = nn.Linear(1024, out_size)
        self.fc22 = nn.Linear(1024, out_size)
        self.softplus = nn.Softplus()
    
    def forward(self, img):
        img = img.reshape(-1, self.img_size[0] * self.img_size[1])
        hidden = F.relu(self.fc0(img))
        hidden = self.fc1(hidden)
        # mean of noise, used in normal distribution
        mean =  torch.tanh(self.fc21(F.relu(hidden)))
        # std used in normal distribution
        sigma = self.softplus(self.fc22(F.relu(hidden)))
        return mean, sigma

In [7]:
class NumNet(nn.Module):
    def __init__(self, img_size, out_size = 3):
        """
        Network for learning N, number of letters in a captcha
        """
        super(NumNet, self).__init__()
        self.neural_net = nn.Sequential(
            nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1] * 2),
            nn.ReLU(),
            nn.Linear(img_size[0] * img_size[1] * 2, 256),
            nn.ReLU(),
            nn.Linear(256, out_size),
            nn.LogSoftmax(dim=1))
  
    def forward(self, img):
        img = torch.reshape(img, (img.shape[0], img.shape[1] * img.shape[2]))
        prob = self.neural_net(img)
        return prob

In [9]:
class CharNetSingle(nn.Module):
    def __init__(self, img_size, output_size):
        """
        Network for letters in a captcha, given the noise and number of letters
        """
        super(CharNetSingle, self).__init__()
        self.img_size = img_size
        
        # observe layers
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.pfc1 = nn.Linear(8832, 2048)
        self.pfc2 = nn.Linear(2048, 1024)
        self.pfc3 = nn.Linear(1024, 1024)
        self.pfc4 = nn.Linear(1024, output_size)
        self.convBN1 = nn.BatchNorm2d(32)
        self.convBN2 = nn.BatchNorm2d(64)
        #self.c_0 = nn.Parameter(torch.zeros(self.num_layers, 1, self.hidden_size)) # define if use LSTM

    def forward(self, img):
        BATCH_SIZE = img.shape[0]
        # observe layers
        img = torch.reshape(img, (BATCH_SIZE, 1, self.img_size[0], self.img_size[1]))

        img = self.pool(F.relu(self.convBN1(self.conv1(img))))
        img = self.pool(F.relu(self.convBN2(self.conv2(img))))
        
        img = torch.reshape(img, (BATCH_SIZE, 8832))

        img = F.relu(self.pfc1(img))
        
        img = F.relu(self.pfc2(img))
        img = F.relu(self.pfc3(img))
        img = self.pfc4(img)
        
        outputs = F.log_softmax(img, dim=1)
        
        return outputs

In [10]:
def inference(t, use_cuda=False):
    """
    one epoch of inference (iterate the training set once)
    """
    loss = 0
    length = TrainingSample
    loss_group = []
    all_batches = make_batches(TrainLoaders)
    for i_batch, sample_batched in enumerate(all_batches):
        
        img = sample_batched[1]
        if use_cuda:
            img = img.cuda()
        IMG = {"captcha" : img}
        imme_loss = csis.step(observations=IMG)
        loss += imme_loss / length

    print("loss at epoch {} is {}".format(t, loss))
    return loss

In [11]:
def test(n = 0, use_train=False, verbose=False, use_cuda=False):
    """
    benchmarking performance on customized or training set
    """
    if use_train:
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=captcha_generated)
    else:
        test_captcha_generated = generate_random_captcha(n, save=False)
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=test_captcha_generated)
    
    total_correct = 0
    char_correct = 0
    total_char = 0
    all_batches = make_batches(TestLoaders)
    for i_batch, t in enumerate(all_batches):

        label = t[0][0]
        gt_noise = t[2][0]
        img = t[1]

        if use_cuda:
            img = img.cuda()
        
        IMG = {"captcha" : img}
        
        posterior = csis.run(observations=IMG)
        marginal_num = pyro.infer.EmpiricalMarginal(posterior, "num_char")
        marginal_noise = pyro.infer.EmpiricalMarginal(posterior, "noise")
        with torch.no_grad():

            N_index = marginal_num()
            N = N_index + captchaModel.num_char_domain[0]
            noise = captchaModel._map_to_noise_range(marginal_noise()[0])
            sampled_chars = []
            
            # sample characters one by one
            for i in range(N):
                marginal_char = pyro.infer.EmpiricalMarginal(posterior, "char_{}".format(i))()[0]
                if use_cuda:
                    marginal_char.cpu()
                sampled_chars.append(marginal_char)
        
        chars = ""
        for i in range(len(sampled_chars)):
            c = sampled_chars[i]
            chars +=  captchaModel.char_dict[c]
        correct = 0
        
        for p_char, t_char in zip(chars, label):
            if p_char == t_char:
                correct += 1
        if not verbose:
            print("N_predicted:", int(N), "| Actual N:", len(label), "| Predicted Noise:", round(float(noise), 3), "| Actual Noise:", round(float(gt_noise), 3), "| Predicted Text:", chars, "| Actual Text:", label, "| Correct:", correct)
        if correct == len(label) and int(N) == len(label):
            total_correct += 1
        char_correct += correct
        total_char += len(label)
    num_test_samples = i_batch + 1
    accuracy = total_correct / num_test_samples
    char_accuracy = char_correct / total_char
    print("use_train =", use_train, "Total correct:", total_correct, "accuracy:{}/{}=".format(total_correct, num_test_samples), accuracy, "char_accuracy:{}/{}=".format(char_correct, total_char), char_accuracy)


In [12]:
def test_cycle(use_cuda):
    
    # disable dropout
    #captchaModel.numNet.eval()
    #captchaModel.charNetSingle.eval()
    test(use_train=True, verbose=True, use_cuda=use_cuda)
    test(1000, use_train=False, verbose=True, use_cuda=use_cuda)
    #test(10, use_train=True, verbose=False, use_cuda=use_cuda)
    test(10, use_train=False, verbose=False, use_cuda=use_cuda)
    # enable dropout
    #captchaModel.numNet.train()
    #captchaModel.charNetSingle.train()

def optimize(use_cuda=False):
    """
    Training/Inferencing Stage
    """
    loss_sequence = []
    pause = 5
    print("Optimizing...")
    for t in range(1, num_steps + 1):
        L = inference(t, use_cuda)
        loss_sequence.append(L)
        if (t % pause == 0) and (t > 0):
            test_cycle(use_cuda=use_cuda)
    plt.plot(loss_sequence)
    plt.title("loss")
    plt.show()

In [13]:
class CaptchaModel(nn.Module):
    """
    network, model and guide wrapper class
    """
    def __init__(self, use_cuda=False):
        super().__init__()
        self.num_char_domain = torch.arange(MIN_N, MAX_N + 1)
        if use_cuda:
            self.num_char_domain = self.num_char_domain.cuda()

        self.numNet = NumNet((captchaHeight, captchaWidth), len(self.num_char_domain))
        self.noiseNet = NoiseNet((captchaHeight, captchaWidth), 1)
        self.char_dict = char_dict # letter dictionary
        self.rnn_hidden_size = 512
        self.rnn_num_layer = 2
        self.charNets = nn.ModuleList([CharNetSingle((captchaHeight, captchaWidth), len(self.char_dict)) for i in range(MAX_N)])
        self.noise_constraint = torch.distributions.constraints.interval(MIN_NOISE, MAX_NOISE)
        if use_cuda:
            self.cuda()
        self.use_cuda = use_cuda
    
    def _map_to_noise_range(self, input):
        """
        map input number to the valid noise range
        """
        input = torch.distributions.transform_to(self.noise_constraint)(input)
        return input

    def guide(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        pyro.module("guide", self)
        img = observations["captcha"].float()
        
        # posterior to the number of letters
        prob = self.numNet(img)
        prob = torch.mean(prob, dim=0)
        N_index = pyro.sample("num_char", dist.Categorical(prob))
        N_index = torch.add(N_index, self.num_char_domain[0])
        
        with pyro.plate("data", img.shape[0]):
            
            # posterior to the noise
            noise_mean, noise_sig = self.noiseNet(img)
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            # posterior to the characters, given number of letters and noise
            for i in range(N_index):
                charP_i = self.charNets[i](img)
                pyro.sample("char_{}".format(i), dist.Categorical(charP_i).to_event(0))
    
    def model(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        
        BS = observations["captcha"].shape[0]
        
        num_p = torch.tensor(1 / len(self.num_char_domain)).repeat(len(self.num_char_domain))
        
        if self.use_cuda:
            num_p = num_p.cuda()
        
        # sample the number of characters
        N_index = pyro.sample("num_char", dist.Categorical(num_p))
        N_index = torch.add(N_index,  self.num_char_domain[0])
        
        with pyro.plate("data", BS):
            
            noise_mean = torch.tensor((MAX_NOISE - MIN_NOISE) / 2).repeat((BS, 1))
            noise_sig = torch.tensor(0.5).repeat((BS, 1))

            if self.use_cuda:
                noise_mean = noise_mean.cuda()
                noise_sig = noise_sig.cuda()

            # sample the noise
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            sampled_c = []
            for i in range(int(N_index)):
                num_c_i = torch.tensor(1 / len(self.char_dict)).repeat((BS, len(self.char_dict)))
                if self.use_cuda:
                    num_c_i = num_c_i.cuda()
                c_i = pyro.sample("char_{}".format(i), dist.Categorical(num_c_i).to_event(0))
                sampled_c.append(c_i)
            
            # sample the character indices
            #c = pyro.sample("chars", dist.Categorical(num_c).to_event(1)) # maybe 2 here
            rendered_images = []
            for i in range(sampled_c[0].shape[0]):
                chars = ""
                for j in range(N_index):
                    #print("sampled_c[%d][%d]" %(j, i), sampled_c[j][i])
                    chars += self.char_dict[sampled_c[j][i]]
        
                rendered_image = render_image(chars, noise=float(noise_batch[i]), use_cuda=self.use_cuda)
                rendered_images.append(rendered_image)
                
        rendered_images = torch.stack(rendered_images)
        sigma = torch.tensor(0.000001)
        if self.use_cuda:
                sigma = sigma.cuda()

        pyro.sample("captcha", dist.Normal(rendered_images, sigma).to_event(2), obs=observations["captcha"])

captchaModel = CaptchaModel(USE_CUDA)

optimiser = pyro.optim.Adam({'lr': 5e-5})
csis = pyro.infer.CSIS(captchaModel.model, captchaModel.guide, optimiser, num_inference_samples=1)


optimize(USE_CUDA)
test_cycle(USE_CUDA)

Optimizing...
loss at epoch 1 is 5.864236591804023
loss at epoch 2 is 2.655128211259838
loss at epoch 3 is 1.5067662035636602
loss at epoch 4 is 0.9278489733896859
loss at epoch 5 is 0.6223891073739921
use_train = True Total correct: 5083 accuracy:5083/10000= 0.5083 char_accuracy:33589/39939= 0.8410075364931521
use_train = False Total correct: 494 accuracy:494/1000= 0.494 char_accuracy:3355/4016= 0.8354083665338645
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.592 | Actual Noise: 0.665 | Predicted Text: 112 | Actual Text: 116 | Correct: 2
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.399 | Actual Noise: 0.345 | Predicted Text: 4903 | Actual Text: 4900 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.334 | Actual Noise: 0.312 | Predicted Text: 1495 | Actual Text: 7095 | Correct: 2
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.575 | Actual Noise: 0.566 | Predicted Text: 3207 | Actual Text: 3607 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.432 |

N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.347 | Actual Noise: 0.048 | Predicted Text: 397 | Actual Text: 397 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.381 | Actual Noise: 0.441 | Predicted Text: 2933 | Actual Text: 2933 | Correct: 4
use_train = False Total correct: 8 accuracy:8/10= 0.8 char_accuracy:37/40= 0.925
loss at epoch 26 is -0.44885624303411376
loss at epoch 27 is -0.45173481781222136
loss at epoch 28 is -0.48568651916354144
loss at epoch 29 is -0.49875285575870865
loss at epoch 30 is -0.49128562221091987
use_train = True Total correct: 8786 accuracy:8786/10000= 0.8786 char_accuracy:38656/39939= 0.9678760109166479
use_train = False Total correct: 885 accuracy:885/1000= 0.885 char_accuracy:3900/4022= 0.9696668324216807
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.316 | Actual Noise: 0.209 | Predicted Text: 17605 | Actual Text: 17605 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.856 | Actual Noise: 0.816 | Predicted Text: 779 | A

N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.675 | Actual Noise: 0.652 | Predicted Text: 623 | Actual Text: 623 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.302 | Actual Noise: 0.231 | Predicted Text: 537 | Actual Text: 537 | Correct: 3
use_train = False Total correct: 10 accuracy:10/10= 1.0 char_accuracy:42/42= 1.0
loss at epoch 51 is -0.7743394207553018
loss at epoch 52 is -0.7892172953131495
loss at epoch 53 is -0.7861641307044002
loss at epoch 54 is -0.7978813620554551
loss at epoch 55 is -0.8108312112686051
use_train = True Total correct: 9289 accuracy:9289/10000= 0.9289 char_accuracy:39200/39939= 0.981496782593455
use_train = False Total correct: 934 accuracy:934/1000= 0.934 char_accuracy:3947/4016= 0.9828187250996016
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.658 | Actual Noise: 0.668 | Predicted Text: 46570 | Actual Text: 46570 | Correct: 5
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.313 | Actual Noise: 0.141 | Predicted Text: 2248 | Actual T

N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.516 | Actual Noise: 0.569 | Predicted Text: 722 | Actual Text: 725 | Correct: 2
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.305 | Actual Noise: 0.21 | Predicted Text: 83791 | Actual Text: 83791 | Correct: 5
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.456 | Actual Noise: 0.449 | Predicted Text: 5991 | Actual Text: 5991 | Correct: 4
use_train = False Total correct: 9 accuracy:9/10= 0.9 char_accuracy:41/42= 0.9761904761904762
loss at epoch 76 is -0.999315381130809
loss at epoch 77 is -0.9927251930391217
loss at epoch 78 is -1.0175190789210482
loss at epoch 79 is -1.002878940342856
loss at epoch 80 is -1.0088461996162263
use_train = True Total correct: 9477 accuracy:9477/10000= 0.9477 char_accuracy:39393/39939= 0.986329151956734
use_train = False Total correct: 955 accuracy:955/1000= 0.955 char_accuracy:3971/4018= 0.9883026381284221
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.652 | Actual Noise: 0.645 | Predicted Text: 914

KeyboardInterrupt: 